# Functions used for pre-porcessing data

In [5]:
import pandas as pd
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt

### Functions used for pre-processing data

In [2]:
def Match_SurfacePoint_to_CenterlinePoint(Cor_SP,Cor_CLP,max_iter_SP):
    ### take two coordinates dataframe, the first containing the coordintes of the surface points
    ### The second for the coordinates of the centerline points. 
    ### This function will then calculated the euclidian distance between the points. 
    ### Only keeping the smalles values. The output consist out of the index of the surfacepoint
    ### and the corresponding centerlinepoint
 
    #timer
    #start_time = time.time()
    
    #convert input values (dataframe) to array values
    arr_SP = Cor_SP.values
    arr_CLP = Cor_CLP.values
    
    # initial d_min is an arbitrary chosen value, 
    d_min = 50

    # empty list to store the data
    min_array = []
    index_array = []
    point_vector_array = []
    
    #row counter to find the index of the smallest point
    print_counter = 0
    Nloops = len(arr_SP)
    max_iter_SP_ = max_iter_SP
    
    # loop through and calculate all distances from surface points to centerlines (for every surface point).
    # save the shortage distance and index of that cell
    for row_SP in arr_SP:
        
        if ((print_counter % 10000)==0):
            print("calculating (%): " + str(round((100*(print_counter/Nloops)),1)) + '(' + str(print_counter) +' out of ' + str(Nloops)+')')

        #reset index & d_min for each point
        index = 0
        d_min = 10
        max_cal_counter = 0
        
        for row_CLP in arr_CLP:
            point_vector = (row_SP - row_CLP)
            #calculate the magnitude of the vector
            d = np.linalg.norm(point_vector)

            # store the smallest distance and index of point centerline
            if d < d_min:
                d_min = d
                row_index = index
                point_vector_data = list(point_vector)
            if d_min == d_min:
                max_cal_counter = max_cal_counter+1
                
            if max_cal_counter > max_iter_SP_:
                max_cal_counter = 0
                next

            index = index +1
            
        print_counter = print_counter +1
        min_array.append(d_min)
        index_array.append(row_index)
        point_vector_array.append(point_vector_data)
        


    a = list(zip(index_array,min_array,point_vector_array))
    df_results = pd.DataFrame(a,columns = ['centerline_point_index','min_distance','point_vector'])
    # print("--- %s seconds ---" % (time.time() - start_time))
    return(df_results)
    

In [3]:
def merge_df_SP_CLP_Match_cal(df_SP, df_CLP,df_SP_CLP_calculation):
    #rename columns to identify them in the final dataframe
    df_SP = df_SP.rename(columns={'X':'X_SP','Y':'Y_SP','Z':'Z_SP'})
    df_CLP = df_CLP.rename(columns={'X':'X_CLP','Y':'Y_CLP','Z':'Z_CLP'})
    
    #two step merge of the three dataframes
    df_merge1 = df_SP_CLP_calculation.merge(df_CLP,left_on=['centerline_point_index'],right_index=True)
    df_merge2 = df_SP.merge(df_merge1,left_index=True, right_index=True)
    return(df_merge2)
    


In [3]:
def Calculate_Surface_area(df_merged):
    #sort the dataframe first
    df_merged = df_merged.sort_values(['centerline_point_index','AngularMetric'],ascending=[True, False])
    
    # calculate the surface area for each centerlinepoint
    area_list = []
    Loop_list = df_merged['centerline_point_index'].unique()
    #Loop through all surface points which have a centerline assigned
    for i in Loop_list:
        #filter the dataframe for each centerline_point
        df_merged[df_merged['centerline_point_index']== i]
        Area = 0
        #get the amount of surface points assinged to a specifc centerline_point_index
        N_tri_edg = len(df_merged[df_merged['centerline_point_index']== i])
        print(N_tri_edg)
        for j in range(0,N_tri_edg):
            #check if j is last point, then cross last with first point
            if j == N_tri_edg:
                vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                vec_B = np.array(literal_eval(df_merged['point_vector'][0]))
                Area = Area + 0.5 * np.linalg.norm(np.cross(vec_A,vec_B))

            if j < N_tri_edg:
                vec_A = np.array(literal_eval(df_merged['point_vector'][j]))
                vec_B = np.array(literal_eval(df_merged['point_vector'][j+1]))
                Area = Area + 0.5 * np.linalg.norm(np.cross(vec_A,vec_B))

        area_list.append([Area,i])
    df_area = pd.DataFrame(area_list,  columns=['Area', 'index_CL'])   
    return(df_area)

In [43]:
def Calculate_centerline_length(df_CLP):
    vec= np.zeros(len(df_CLP))
    CLP_cor = df_CLP[['X','Y','Z']]
    for i in range(1,len(df_CLP)):        
        CLP_len = CLP_cor.iloc[i] - CLP_cor.iloc[i-1]
        vec[i] = np.linalg.norm(CLP_len)
    distances_CLP = np.cumsum(vec)
    return(distances_CLP)
